In [1]:
# Imports library
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import os
import collections

from tensor2tensor import models
from tensor2tensor import problems
from tensor2tensor.layers import common_layers
from tensor2tensor.utils import trainer_lib
from tensor2tensor.utils import t2t_model
from tensor2tensor.utils import registry
from tensor2tensor.utils import metrics


/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(

In [2]:
# Enable TF Eager execution
tfe = tf.contrib.eager
tfe.enable_eager_execution()

# Other setup
Modes = tf.estimator.ModeKeys

# Setup some directories
data_dir = os.path.expanduser("/t2tdata")
tmp_dir = os.path.expanduser("/t2tdata")
train_dir = os.path.expanduser("/model_files")

checkpoint_dir = os.path.expanduser("/checkpoints")
tf.gfile.MakeDirs(data_dir)
tf.gfile.MakeDirs(tmp_dir)
tf.gfile.MakeDirs(train_dir)
tf.gfile.MakeDirs(checkpoint_dir)
gs_data_dir = "gs://tensor2tensor-data"
gs_ckpt_dir = "gs://tensor2tensor-checkpoints/"




# # Enable TF Eager execution
# tfe = tf.contrib.eager
# tfe.enable_eager_execution()

# # Other setup
# Modes = tf.estimator.ModeKeys

# # Setup some directories
# data_dir = os.path.expanduser("/t2tdata")
# tmp_dir = os.path.expanduser("/t2tdata")
# train_dir = os.path.expanduser("/model_files")

# ende_problem = problems.problem("summarize_cnn_dailymail32k")
# vocab_name = "vocab.summarize_cnn_dailymail32k.32768.subwords"
# vocab_file = os.path.join(gs_data_dir, vocab_name)

# checkpoint_dir = os.path.expanduser("/model_files")
# tf.gfile.MakeDirs(data_dir)
# tf.gfile.MakeDirs(tmp_dir)
# tf.gfile.MakeDirs(train_dir)
# tf.gfile.MakeDirs(checkpoint_dir)
# gs_data_dir = "gs://tensor2tensor-data"
# gs_ckpt_dir = "gs://tensor2tensor-checkpoints/"



# ende_problem.generate_data(data_dir, tmp_dir) 


In [3]:
problems.available()

['algorithmic_addition_binary40',
 'algorithmic_addition_decimal40',
 'algorithmic_cipher_shift200',
 'algorithmic_cipher_shift5',
 'algorithmic_cipher_vigenere200',
 'algorithmic_cipher_vigenere5',
 'algorithmic_identity_binary40',
 'algorithmic_identity_decimal40',
 'algorithmic_math_two_variables',
 'algorithmic_multiplication_binary40',
 'algorithmic_multiplication_decimal40',
 'algorithmic_reverse_binary40',
 'algorithmic_reverse_binary40_test',
 'algorithmic_reverse_decimal40',
 'algorithmic_reverse_nlplike32k',
 'algorithmic_reverse_nlplike8k',
 'algorithmic_shift_decimal40',
 'algorithmic_sort_problem',
 'audio_timit_characters_tune',
 'audio_timit_tokens8k_test',
 'audio_timit_tokens8k_tune',
 'babi_qa_concat_all_tasks_10k',
 'babi_qa_concat_all_tasks_1k',
 'babi_qa_concat_task10_10k',
 'babi_qa_concat_task10_1k',
 'babi_qa_concat_task11_10k',
 'babi_qa_concat_task11_1k',
 'babi_qa_concat_task12_10k',
 'babi_qa_concat_task12_1k',
 'babi_qa_concat_task13_10k',
 'babi_qa_concat_

In [ ]:
# Fetch the problem
ende_problem = problems.problem("summarize_cnn_dailymail32k")

# # Copy the vocab file locally so we can encode inputs and decode model outputs
# # All vocabs are stored on GCS
# vocab_name = "vocab.summarize_cnn_dailymail32k.32768.subwords"
# vocab_file = os.path.join(gs_data_dir, vocab_name)
# !gsutil cp {vocab_file} {data_dir}

# t2t_problem = problems.problem(PROBLEM)
ende_problem.generate_data(data_dir, tmp_dir) 




# Get the encoders from the problem
encoders = ende_problem.feature_encoders(data_dir)

# Setup helper functions for encoding and decoding
def encode(input_str, output_str=None):
    """Input str to features dict, ready for inference"""
    inputs = encoders["inputs"].encode(input_str) + [1]  # add EOS id
    batch_inputs = tf.reshape(inputs, [1, -1, 1])  # Make it 3D.
    return {"inputs": batch_inputs}

def decode(integers):
    """List of ints to str"""
    integers = list(np.squeeze(integers))
    if 1 in integers:
        integers = integers[:integers.index(1)]
    return encoders["inputs"].decode(np.squeeze(integers))





# # # Fetch the problem
# # ende_problem = problems.problem("translate_ende_wmt32k")

# # # Copy the vocab file locally so we can encode inputs and decode model outputs
# # # All vocabs are stored on GCS
# # vocab_name = "vocab.translate_ende_wmt32k.32768.subwords"
# # vocab_file = os.path.join(gs_data_dir, vocab_name)
# # !gsutil cp {vocab_file} {data_dir}

# # Get the encoders from the problem
# encoders = ende_problem.feature_encoders(data_dir)

# # Setup helper functions for encoding and decoding
# def encode(input_str, output_str=None):
#     """Input str to features dict, ready for inference"""
#     inputs = encoders["inputs"].encode(input_str) + [1]  # add EOS id
#     batch_inputs = tf.reshape(inputs, [1, -1, 1])  # Make it 3D.
#     return {"inputs": batch_inputs}

# def decode(integers):
#     """List of ints to str"""
#     integers = list(np.squeeze(integers))
#     if 1 in integers:
#         integers = integers[:integers.index(1)]
#     return encoders["inputs"].decode(np.squeeze(integers))

INFO:tensorflow:Not downloading, file already found: /t2tdata/all_train.txt
INFO:tensorflow:Found vocab file: /t2tdata/vocab.summarize_cnn_dailymail32k.32768.subwords
INFO:tensorflow:Not downloading, file already found: /t2tdata/all_train.txt
INFO:tensorflow:Writing cnndm.train
INFO:tensorflow:Found 287227 examples
INFO:tensorflow:Found 287227 examples
INFO:tensorflow:Generating case 0.
INFO:tensorflow:Generating case 100000.
INFO:tensorflow:Generating case 200000.


In [ ]:
# Create hparams and the model
model_name = "transformer"
hparams_set = "transformer_base"

hparams = trainer_lib.create_hparams(hparams_set, data_dir=data_dir, problem_name="summarize_cnn_dailymail32k")

# NOTE: Only create the model once when restoring from a checkpoint; it's a
# Layer and so subsequent instantiations will have different variable scopes
# that will not match the checkpoint.
translate_model = registry.model(model_name)(hparams, Modes.EVAL)



# # Create hparams and the model
# model_name = "transformer"
# hparams_set = "transformer_base"

# hparams = trainer_lib.create_hparams(hparams_set, data_dir=data_dir, problem_name="translate_ende_wmt32k")

# # NOTE: Only create the model once when restoring from a checkpoint; it's a
# # Layer and so subsequent instantiations will have different variable scopes
# # that will not match the checkpoint.
# translate_model = registry.model(model_name)(hparams, Modes.EVAL)

In [ ]:
# Copy the pretrained checkpoint locally
ckpt_name = "transformer_ende_test"
gs_ckpt = os.path.join(gs_ckpt_dir, ckpt_name)
!gsutil -q cp -R {gs_ckpt} {checkpoint_dir}
ckpt_path = tf.train.latest_checkpoint(os.path.join(checkpoint_dir, ckpt_name))
ckpt_path

# # Copy the pretrained checkpoint locally
# ckpt_name = "transformer_ende_test"
# gs_ckpt = os.path.join(gs_ckpt_dir, ckpt_name)
# # !gsutil -q cp -R {gs_ckpt} {checkpoint_dir}
# ckpt_path = tf.train.latest_checkpoint(os.path.join(checkpoint_dir, ckpt_name))
# ckpt_path

In [ ]:
# Restore and translate!
def translate(inputs):
    encoded_inputs = encode(inputs)
    with tfe.restore_variables_on_create(ckpt_path):
        model_output = translate_model.infer(encoded_inputs)["outputs"]
    return decode(model_output)

inputs = "I havent slept for 10 days because that would be too long."
outputs = translate(inputs)

print("Inputs: %s" % inputs)
print("Outputs: %s" % outputs)


# # Restore and translate!
# def translate(inputs):
#     encoded_inputs = encode(inputs)
#     with tfe.restore_variables_on_create(ckpt_path):
#         model_output = translate_model.infer(encoded_inputs)["outputs"]
#     return decode(model_output)

# inputs = "The animal didn't cross the street because it was too tired"
# outputs = translate(inputs)

# print("Inputs: %s" % inputs)
# print("Outputs: %s" % outputs)
